# CSMCC16 Flight Coursework

### Import packages

In [479]:
import numpy as np
import pandas as pd
import datetime
import re
from difflib import SequenceMatcher

### Import data

In [480]:
#import csv - no error was used when building initial solution
#pass_data = pd.read_csv('AComp_Passenger_data_no_error.csv',header=None)   #comment out as necessary
pass_data = pd.read_csv('AComp_Passenger_data.csv',header=None)   #comment out as necessary
pass_datet = pd.read_csv('AComp_Passenger_data_no_error_DateTime.csv',header=None)
airport_lat_lon = pd.read_csv('Top30_airports_LatLong.csv',header=None)

In [481]:
#check data
#pass_data

In [482]:
# Create as dataframes
pass_data = pd.DataFrame(pass_data)
pass_datetime = pd.DataFrame(pass_datet)
airport_lat_lon = pd.DataFrame(airport_lat_lon)

In [483]:
# Add Column Names
pass_data.columns = ['Passenger_id'
                        ,'Flight_id'
                        ,'From_airport_IATA'
                        ,'Dest_airport_IATA'
                        ,'Depart_time_epoch'
                        ,'Flight_time_min']
airport_lat_lon.columns = ['Airport_name'
                        ,'Airport_IATA_code'
                        ,'Latitude'
                        ,'Longitude']
pass_datetime.columns = ['Passenger_id'
                        ,'Flight_id'
                        ,'From_airport_IATA'
                        ,'Dest_airport_IATA'
                        ,'Departure_date_time'
                        ,'Depart_time_epoch'
                        ,'Flight_time_min'
                        ,'Arrive_date_time'
]

### Remove missing values

In [484]:
#Remove values where passenger id and flight id are missing
pass_data=pass_data.dropna(subset=['Flight_id','Passenger_id'])

### Additional Columns and formats

In [485]:
#Calculate epoch Arrival Time
pass_data['Arrive_epoch'] = pass_data['Depart_time_epoch']+pass_data['Flight_time_min']*60
#Convert epoch to datetime
pass_data['Depart_time_datetime'] = pd.to_datetime(pass_data['Depart_time_epoch'],unit='s')
pass_data['Arrive_time_datetime'] = pd.to_datetime(pass_data['Arrive_epoch'],unit='s')

In [486]:
#And new datetime formats
pass_data['Depart_day_month']=pass_data['Depart_time_datetime'].dt.strftime("%a %d/%b/%Y")
#pass_data_df['Depart_day_datetime']=pass_data_df['Depart_time_datetime'].dt.strftime("%a %d/%b/%Y %H:%M:%S")
pass_data['Depart_time']=pass_data['Depart_time_datetime'].dt.strftime("%H:%M:%S")
pass_data['Arrive_time']=pass_data['Arrive_time_datetime'].dt.strftime("%H:%M:%S")

In [487]:
#Add column to allow counting of passengers
pass_data['count']='1'

In [488]:
# Check data
print(pass_data)
print(pass_datetime)
print(airport_lat_lon)

    Passenger_id Flight_id From_airport_IATA Dest_airport_IATA  \
0     UES9151GS5  SQU6245R               DEN               FRA   
1     UES9151GS5  XXQ4064B               JFK               FRA   
2     EZC9678QI6  SOH3431A               ORD               MIA   
3     ONL0812DH1  SOH3431A               ORD               MIA   
4     CYJ0225CH1  PME8178S               DEN               PEK   
..           ...       ...               ...               ...   
504   BWI0520BG6  BER7172M               KUL               LAS   
505   LLZ3798PE3  EWH6301Y               CAN               DFW   
506   KKP5277HZ7  KJR6646J               IAH               BKK   
507   JJM4724RF7  XXQ4064B               JFK               FRA   
508   SJD8775RZ4  WSK1289Z               CLT               vEN   

     Depart_time_epoch  Flight_time_min  Arrive_epoch Depart_time_datetime  \
0           1420564460             1049    1420627400  2015-01-06 17:14:20   
1           1420563917              802    14206120

### Flight data only

In [489]:
#select subset of data 
flight_only_data = pass_data[['Flight_id'
                      ,'From_airport_IATA'
                      ,'Dest_airport_IATA'
                      ,'Depart_day_month'
                      ,'Depart_time'         
                      ,'Flight_time_min'
                      ,'count'
                        ]]

In [490]:
#Group data by flight
flight_only_data = flight_only_data.groupby(['Flight_id'
                          ,'From_airport_IATA'
                      ,'Dest_airport_IATA'
                      ,'Depart_day_month'
                      ,'Depart_time'         
                      ,'Flight_time_min'           
                         ]).count()
#reset index
flight_only_data.reset_index(inplace=True)

#checks
flight_only_data
len(flight_only_data)


54

## Error Correction
### Flights

In [491]:
#check data
flight_only_data

,Flight_id,From_airport_IATA,Dest_airport_IATA,Depart_day_month,Depart_time,Flight_time_min,count
0,4QU6245R,DEN,FRA,Tue 06/Jan/2015,17:14:20,1049,1
1,ATT7791R,A;S,DEN,Tue 06/Jan/2015,17:13:14,1001,1
2,ATT7791R,AM%,DEN,Tue 06/Jan/2015,17:13:14,1001,1
3,ATT7791R,AMS,DEN,Tue 06/Jan/2015,17:13:14,1001,13
4,BER7172M,KUL,LAS,Tue 06/Jan/2015,17:26:07,1848,16
5,BER7172M,KUL,[AS,Tue 06/Jan/2015,17:26:07,1848,1
6,DAU2617A,CGK,SFO,Tue 06/Jan/2015,17:23:06,1811,12
7,DKZ3042O,MIA,SFO,Tue 06/Jan/2015,17:05:27,538,11
8,EWH6301Y,CAN,DFW,Tue 06/Jan/2015,17:22:47,1683,9
9,EWH6301Y,~AN,DFW,Tue 06/Jan/2015,17:22:47,1683,1


In [492]:
#correct formats
format_IATA = r"[A-Z][A-Z][A-Z]"
format_flight_id = r"[A-Z][A-Z][A-Z][0-9][0-9][0-9][0-9][A-Z]"

#format_pass_id = r"[A-Z][A-Z][A-Z][0-9][0-9][0-9][0-9][A-Z][A-Z][0-9]"
#flight_only_data['Pass_id_format'] =flight_only_data['Passenger_id'].str.contains(format_pass_id)

#add column indicating if correct
flight_only_data['From_airport_format'] =flight_only_data['From_airport_IATA'].str.contains(format_IATA)
flight_only_data['Dest_airport_format'] =flight_only_data['Dest_airport_IATA'].str.contains(format_IATA)
flight_only_data['Flight_id_format'] =flight_only_data['Flight_id'].str.contains(format_flight_id)

In [493]:
#add column indicating if all flight regex correct
flight_only_data['Flight_info_correct']=flight_only_data.all(axis='columns')

In [494]:
#select only those whose regex is correct
flight_info_true = flight_only_data.loc[flight_only_data['Flight_info_correct'] == True]

In [495]:
#Take True airport values, change column names to enable merge

true_airports_from = pd.DataFrame(airport_lat_lon.Airport_IATA_code.unique())
#rename columns to allow merge
true_airports_from.columns = ['From_airport_IATA']
#add column to know if exists
true_airports_from['From_airport_exists']=True

true_airports_dest = pd.DataFrame(airport_lat_lon.Airport_IATA_code.unique())
#rename columns to allow merge
true_airports_dest.columns = ['Dest_airport_IATA']
#add column to know if exists
true_airports_dest['Dest_airport_exists']=True

In [496]:
#send to cv for checking
#merge_with_dest.to_csv(r'merge_with_dest.csv')

In [497]:
#merge with from
merge_with_from=flight_info_true.merge(true_airports_from
                     , how='outer', on=['From_airport_IATA'])
#merge with dest
merge_with_dest=merge_with_from.merge(true_airports_dest
                     , how='outer', on=['Dest_airport_IATA'])
#check data
#merge_with_dest

In [498]:
#drop NaNs - any that are not in the true airport list
merge_with_dest=merge_with_dest.dropna(subset=['From_airport_exists', 'Dest_airport_exists'])

In [499]:
merge_with_dest

,Flight_id,From_airport_IATA,Dest_airport_IATA,Depart_day_month,Depart_time,Flight_time_min,count,From_airport_format,Dest_airport_format,Flight_id_format,Flight_info_correct,From_airport_exists,Dest_airport_exists
0,ATT7791R,AMS,DEN,Tue 06/Jan/2015,17:13:14,1001.0,13.0,True,True,True,True,True,True
1,WSK1289Z,CLT,DEN,Tue 06/Jan/2015,16:59:02,278.0,20.0,True,True,True,True,True,True
2,BER7172M,KUL,LAS,Tue 06/Jan/2015,17:26:07,1848.0,16.0,True,True,True,True,True,True
3,QHU1140O,CDG,LAS,Tue 06/Jan/2015,17:14:58,1133.0,20.0,True,True,True,True,True,True
4,VDC9164W,FCO,LAS,Tue 06/Jan/2015,17:18:18,1276.0,14.0,True,True,True,True,True,True
5,MBA8071P,KUL,PEK,Tue 06/Jan/2015,17:04:16,572.0,15.0,True,True,True,True,True,True
6,GMO5938W,LHR,PEK,Tue 06/Jan/2015,17:11:57,1057.0,24.0,True,True,True,True,True,True
7,PME8178S,DEN,PEK,Tue 06/Jan/2015,17:13:29,1322.0,17.0,True,True,True,True,True,True
8,PNE8178S,DEN,PEK,Tue 06/Jan/2015,17:13:29,1322.0,1.0,True,True,True,True,True,True
9,WPW9201U,DFW,PEK,Tue 06/Jan/2015,17:21:09,1452.0,10.0,True,True,True,True,True,True


In [500]:
#select subset of data 
flight_only_data = merge_with_dest[['Flight_id'
                      ,'From_airport_IATA'
                      ,'Dest_airport_IATA'
                      ,'Depart_time'
                      ,'count'
                        ]]

In [501]:
#Group data to see total flights
flight_only_data = flight_only_data.groupby(['Flight_id'
                        ,'From_airport_IATA'
                      ,'Dest_airport_IATA'
                      ,'Depart_time'
                         ]).sum()
flight_only_data

,,,,count
Flight_id,From_airport_IATA,Dest_airport_IATA,Depart_time,
ATT7791R,AMS,DEN,17:13:14,13.0
BER7172M,KUL,LAS,17:26:07,16.0
DAU2617A,CGK,SFO,17:23:06,12.0
DKZ3042O,MIA,SFO,17:05:27,11.0
EWH6301Y,CAN,DFW,17:22:47,9.0
FYL5866L,ATL,HKG,17:25:40,20.0
GMO5938W,LHR,PEK,17:11:57,24.0
HUR0974O,DEN,PVG,17:15:25,7.0
HZT2506M,IAH,AMS,17:12:04,13.0


In [502]:
flight_only_data.reset_index(inplace=True)

In [503]:
flight_only_data['Route'] = flight_only_data['From_airport_IATA']+'-' + flight_only_data['Dest_airport_IATA']+'-' + flight_only_data['Depart_time']

In [504]:
#sort, route in alpha-numeric order, count in descending (lowest value last)
flight_only_data.sort_values(by=['Route','count']
                             , ascending=[True,False]
                             , na_position='first')

,Flight_id,From_airport_IATA,Dest_airport_IATA,Depart_time,count,Route
0,ATT7791R,AMS,DEN,17:13:14,13.0,AMS-DEN-17:13:14
5,FYL5866L,ATL,HKG,17:25:40,20.0,ATL-HKG-17:25:40
28,XOY7948U,ATL,LHR,17:07:18,16.0,ATL-LHR-17:07:18
30,YZO4444S,BKK,MIA,17:28:50,16.0,BKK-MIA-17:28:50
4,EWH6301Y,CAN,DFW,17:22:47,9.0,CAN-DFW-17:22:47
21,ULZ8130D,CAN,DFW,17:23:03,27.0,CAN-DFW-17:23:03
15,QHU1140O,CDG,LAS,17:14:58,20.0,CDG-LAS-17:14:58
20,TMV7633W,CGK,DXB,17:05:58,14.0,CGK-DXB-17:05:58
2,DAU2617A,CGK,SFO,17:23:06,12.0,CGK-SFO-17:23:06
26,WSK1289Z,CLT,DEN,16:59:02,20.0,CLT-DEN-16:59:02


In [505]:
# Remove duplicate with lowest value
flight_only_data['Route_dupe']=flight_only_data['Route'].duplicated()
flight_only_deduped = flight_only_data[flight_only_data.Route_dupe == False]

In [506]:
#Add route info to pass_data
pass_data['Route'] = pass_data['From_airport_IATA']+'-' + pass_data['Dest_airport_IATA']+'-' + pass_data['Depart_time']

In [507]:
#Select data required
pass_data_slim = pass_data[['Passenger_id','Flight_id','Route']]
flight_id_route = flight_only_deduped[['Route','Flight_id']]

In [508]:
#Merge correct flight id
flight_id_route.columns = ['Route','correct_Flight_id']
pass_data_route = pass_data_slim.merge(flight_id_route, how='outer', on=['Route'])
pass_data_route.loc[pass_data_route['correct_Flight_id'].notna(), 'Final_Flight_id'] = pass_data_route['correct_Flight_id']
pass_data_route.loc[pass_data_route['correct_Flight_id'].isnull(), 'Final_Flight_id'] = pass_data_route['Flight_id']

In [509]:
#Merge correct route info
flight_id_route.columns = ['Correct_Route','Flight_id']
pass_data_route = pass_data_route.merge(flight_id_route, how='outer', on=['Flight_id'])
pass_data_route.loc[pass_data_route['Correct_Route'].notna(), 'Final_Route'] = pass_data_route['Correct_Route']
pass_data_route.loc[pass_data_route['Correct_Route'].isnull(), 'Final_Route'] = pass_data_route['Route']

In [510]:
pass_data_route = pass_data_route[['Passenger_id','Final_Flight_id','Final_Route']]
pass_data_route.columns = ['Passenger_id','Flight_id','Route']

In [511]:
#pass_data_route.to_csv((r'pass_data_route.csv'))
pass_data_route

,Passenger_id,Flight_id,Route
0,UES9151GS5,SQU6245R,DEN-FRA-17:14:20
1,UES915*GS5,SQU6245R,DEN-FRA-17:14:20
2,JBE2302VO4,SQU6245R,DEN-FRA-17:14:20
3,HCA3158QA6,SQU6245R,DEN-FRA-17:14:20
4,XFG5747ZT9,SQU6245R,DEN-FRA-17:14:20
...,...,...,...
495,HCA3158QA6,XOY7948U,ATL-LHR-17:07:18
496,WBE6935NU$,XOY7948U,ATL-LHR-17:07:18
497,SPR4484HA6,XOY7948U,ATL-LHR-17:07:18
498,VZY2993ME1,XOY7948U,ATL-LHR-17:07:18


## Error Correction
### Passenger

In [478]:
format_pass_id = r"[A-Z][A-Z][A-Z][0-9][0-9][0-9][0-9][A-Z][A-Z][0-9]"
pass_data_route_test=pass_data_route
pass_data_route_test['Pass_id_format'] =pass_data_route['Passenger_id'].str.contains(format_pass_id)
pass_data_route_test

,Passenger_id,Flight_id,Route,Pass_id_format,count
0,UES9151GS5,SQU6245R,DEN-FRA-17:14:20,True,1
1,UES915*GS5,SQU6245R,DEN-FRA-17:14:20,False,1
2,JBE2302VO4,SQU6245R,DEN-FRA-17:14:20,True,1
3,HCA3158QA6,SQU6245R,DEN-FRA-17:14:20,True,1
4,XFG5747ZT9,SQU6245R,DEN-FRA-17:14:20,True,1
...,...,...,...,...,...
495,HCA3158QA6,XOY7948U,ATL-LHR-17:07:18,True,1
496,WBE6935NU$,XOY7948U,ATL-LHR-17:07:18,False,1
497,SPR4484HA6,XOY7948U,ATL-LHR-17:07:18,True,1
498,VZY2993ME1,XOY7948U,ATL-LHR-17:07:18,True,1


In [512]:
#add column to count
pass_data_route_test['count'] = '1'

In [513]:
#select subset of data 
pass_data_route_test = pass_data_route_test[['Passenger_id'
                   #   ,'Flight_id'
                      ,'Pass_id_format'
                      ,'count'
                        ]]

In [514]:
#Group data to see passengers and total flights by passenger id
pass_only=pass_data_route_test.groupby(['Passenger_id'
                        ,'Pass_id_format'
                         ]).count()
#reset index
pass_only.reset_index(inplace=True)

pass_only

,Passenger_id,Pass_id_format,count
0,BWI0520BG6,True,23
1,CDC0302NN5,True,12
2,CKZ3132BR4,True,18
3,CKZ313\BR4,False,1
4,CXN7304ER2,True,17
5,CYJ0225CH1,True,11
6,DAZ3029XA0,True,23
7,EDV2089LK5,True,13
8,EZC9678QI6,True,21
9,HCA3158QA6,True,21


In [515]:
#select only those whose regex is correct 
## this is not necessary as the second code will remove these, 
## however it is left here as this will be a better way to check if correct passenger id
pass_info_true = pass_only.loc[pass_only['Pass_id_format'] == True]

In [516]:
#select only those who have more than 1 flight
pass_info_true = pass_only.loc[pass_only['count'] != 1]
#pass_info_true

In [517]:
pass_info_false = pass_only.loc[pass_only['count'] == 1]
#pass_info_false

In [518]:
#Create list of false passenger ids
false = pass_info_false['Passenger_id']
false

3     CKZ313\BR4
23    SJD8|75RZ4
25    SPRb484HA6
26    UES915*GS5
28    UMH6360YP0
29    VZT2993ME1
31    WBE6935NU$
35    WYu2010YH8
Name: Passenger_id, dtype: object

In [519]:
#Create list of false passenger ids
true = pass_info_true['Passenger_id']
#true

In [520]:
#create function to find the best match for a given false value and list for true values
def best_match(f,t):
    global test_best_true
    global final_true
    test_best_true = []
    final_true =[]
    for t in true:
        test_best_true.append([t,round(SequenceMatcher(None, t, f).ratio(),3)])
    test_best_true = pd.DataFrame(test_best_true)
    max_true = test_best_true.iloc[test_best_true[1].idxmax(), 0]
    final_true.append([f,max_true])
    #final_true = pd.DataFrame (final_true)
   # print(final_true)

In [521]:
#test each false value
best_match_results = []
for f in false:
    best_match(f,t)
    best_match_results.append(final_true)

In [522]:
#turn into dataframe
best_match_results_pd=pd.DataFrame (best_match_results)
best_match_results_pd.columns = ['results']
best_match_results_pd
#obj_1_list.columns = ['From_airport_IATA']

,results
0,"[CKZ313\BR4, CKZ3132BR4]"
1,"[SJD8|75RZ4, SJD8775RZ4]"
2,"[SPRb484HA6, SPR4484HA6]"
3,"[UES915*GS5, UES9151GS5]"
4,"[UMH6360YP0, YMH6360YP0]"
5,"[VZT2993ME1, VZY2993ME1]"
6,"[WBE6935NU$, WBE6935NU3]"
7,"[WYu2010YH8, WYU2010YH8]"


In [530]:
#pass_data_route['From_airport_IATA']
#best_match_results_pd['results'].str.split('-',expand=True)
pass_data_route[['From_airport_IATA','To_airport_IATA','Time']]=pass_data_route['Route'].str.split('-',expand=True)
pass_data_route

,Passenger_id,Flight_id,Route,From_airport_IATA,To_airport_IATA,Time
0,UES9151GS5,SQU6245R,DEN-FRA-17:14:20,DEN,FRA,17:14:20
1,UES915*GS5,SQU6245R,DEN-FRA-17:14:20,DEN,FRA,17:14:20
2,JBE2302VO4,SQU6245R,DEN-FRA-17:14:20,DEN,FRA,17:14:20
3,HCA3158QA6,SQU6245R,DEN-FRA-17:14:20,DEN,FRA,17:14:20
4,XFG5747ZT9,SQU6245R,DEN-FRA-17:14:20,DEN,FRA,17:14:20
...,...,...,...,...,...,...
495,HCA3158QA6,XOY7948U,ATL-LHR-17:07:18,ATL,LHR,17:07:18
496,WBE6935NU$,XOY7948U,ATL-LHR-17:07:18,ATL,LHR,17:07:18
497,SPR4484HA6,XOY7948U,ATL-LHR-17:07:18,ATL,LHR,17:07:18
498,VZY2993ME1,XOY7948U,ATL-LHR-17:07:18,ATL,LHR,17:07:18


## Objective 1: 
### Determine the number of flights from each airport; include a list of any airports not used.

In [469]:
# List of top 30 airports
airport_list = airport_lat_lon['Airport_IATA_code']
airport_list.columns = ['From_airport_IATA']
airport_list = pd.DataFrame(airport_list)

In [472]:
#check data
#airport_list

In [531]:
# Count number of departure flights
from_df = pass_data_route['From_airport_IATA']
from_counts = from_df.value_counts()
from_counts_df = pd.DataFrame(from_counts)
# move index to column
from_counts_df['Airport_IATA_code'] =from_counts_df.index

In [532]:
#Check data
print(from_df)
print(from_counts)
print(from_counts_df)

0      DEN
1      DEN
2      DEN
3      DEN
4      DEN
      ... 
495    ATL
496    ATL
497    ATL
498    ATL
499    ATL
Name: From_airport_IATA, Length: 500, dtype: object
DEN    46
IAH    37
CAN    37
ATL    36
KUL    33
ORD    33
CGK    27
JFK    25
LHR    25
CLT    21
CDG    21
PVG    20
BKK    17
LAS    17
FCO    15
AMS    15
MUC    14
MAD    13
PEK    13
HND    13
MIA    11
DFW    11
Name: From_airport_IATA, dtype: int64
     From_airport_IATA Airport_IATA_code
DEN                 46               DEN
IAH                 37               IAH
CAN                 37               CAN
ATL                 36               ATL
KUL                 33               KUL
ORD                 33               ORD
CGK                 27               CGK
JFK                 25               JFK
LHR                 25               LHR
CLT                 21               CLT
CDG                 21               CDG
PVG                 20               PVG
BKK                 17              

In [534]:
# merge full list of airports with number of flights from each airport
num_flights = airport_list.merge(from_counts_df, how='outer', on=['Airport_IATA_code'])

In [535]:
# clean up results
num_flights=num_flights.fillna(0)
num_flights=num_flights.sort_values(by=['From_airport_IATA','Airport_IATA_code'], ascending=[False,True])
num_flights.set_index('Airport_IATA_code', inplace=True, drop=True)
num_flights

,From_airport_IATA
Airport_IATA_code,
DEN,46.0
CAN,37.0
IAH,37.0
ATL,36.0
KUL,33.0
ORD,33.0
CGK,27.0
JFK,25.0
LHR,25.0


## Objective 2
### Create a list of flights based on the Flight id, this output should include the passenger Id, relevant IATA/FAA codes, the departure time, the arrival time (times to be converted to HH:MM:SS format), and the flight times.

In [ ]:
#select data for objective 2
obj2_data = pass_data_df[['Flight_id'
                      ,'From_airport_IATA'
                      ,'Dest_airport_IATA'
                      ,'Depart_day_month'
                      ,'Depart_time'
                      ,'Arrive_time'
                      ,'Flight_time_min'
                      ,'Passenger_id']]

## Objective 3
### Calculate the number of passengers on each flight.

In [ ]:
obj2_results = obj2_data.groupby(['Flight_id'
                      ,'From_airport_IATA'
                      ,'Dest_airport_IATA'
                      ,'Depart_day_month'
                      ,'Depart_time'
                      ,'Arrive_time'
                      ,'Flight_time_min']).count()
#reset index as columns
obj2_results.reset_index(inplace=True)
obj2_results

## Objective 4
### Calculate the line-of-sight (nautical) miles for each flight and the total travelled by each passenger and thus output the passenger having earned the highest air miles.

In [ ]:
#flight info
flight_from = obj2_results[['Flight_id','From_airport_IATA']]
flight_dest = obj2_results[['Flight_id','Dest_airport_IATA']]

In [ ]:
#rename columns to allow merge
flight_from.columns = ['Flight_id'
                        ,'Airport_IATA_code']
flight_dest.columns = ['Flight_id'
                        ,'Airport_IATA_code']

In [ ]:
#merge from flight with latitude and longitude
from_lat_long = flight_from.merge(airport_lat_lon, how='inner', on=['Airport_IATA_code','Airport_IATA_code'])
from_lat_long.columns = ['Flight_id'
                        ,'From_Airport_IATA_code'
                        ,'From_Airport_Name'
                        ,'From_Lat'
                        ,'From_Lon'
                    ]

In [ ]:
#merge dest flight with latitude and longitude
dest_lat_lon = flight_dest.merge(airport_lat_lon, how='inner', on=['Airport_IATA_code','Airport_IATA_code'])
dest_lat_lon.columns = ['Flight_id'
                        ,'Dest_Airport_IATA_code'
                        ,'Dest_Airport_Name'
                        ,'Dest_Lat'
                        ,'Dest_Lon'
                    ]

In [ ]:
#merge from flight with latitude and longitude
from_dest_lat_lon = from_lat_long.merge(dest_lat_lon, how='outer', on=['Flight_id','Flight_id'])

In [ ]:
from_dest_lat_lon

In [ ]:
# Calculate nautical miles (a^2+b^2=c^2)
from_dest_lat_lon['Miles'] = np.sqrt((from_dest_lat_lon['Dest_Lat']-from_dest_lat_lon['From_Lat'])**2
+(from_dest_lat_lon['Dest_Lon']-from_dest_lat_lon['From_Lon'])**2)

In [ ]:
#export results
from_dest_lat_lon.to_csv(r'from_dest_lat_lon.csv')

In [ ]:
# Number of miles per flight 
miles_flight = from_dest_lat_lon[['Flight_id','Miles']]
miles_flight

In [ ]:
#Add flight miles to passenger data
obj4_data = obj2_data.merge(miles_flight, how='outer', on=['Flight_id', 'Flight_id'])
obj4_data

In [ ]:
#Number of miles per flight per passenger
pass_flights= pd.pivot_table(obj4_data, values='miles', index=['Passenger_id'],
               columns=['Flight_id']
              , aggfunc=np.sum
              )
pass_flights

In [ ]:
pass_flights.to_csv(r'pass_flights.csv')

In [ ]:
#Add up flight miles per passenger
Pass_miles = pass_flights.sum(axis = 1, skipna = True) 
Pass_miles_df = pd.DataFrame(Pass_miles)
Pass_miles_df.columns = ['Air_miles']
Pass_miles_df = Pass_miles_df.sort_values(by=['Air_miles'], ascending=False)
Pass_miles_df.to_csv(r'Pass_miles_df.csv')
Pass_miles_df

# old code, maybe useful

In [ ]:
pass_data_df.value_counts(['From_airport_IATA'])

In [ ]:
pass_data_df.groupby(['From_airport_IATA']).count()

In [ ]:
flight_only_grp.isin(flight_only_grp['From_airport_IATA'])

In [ ]:
#pass_data_df.set_index(["From_airport_IATA"]).count(level="From_airport_IATA")
pass_data_df.count(axis="From_airport_IATA")


In [ ]:
#pd.concat([obj_1_list_df, from_counts_df], keys=['Airport_IATA_code','Airport_IATA_code'])


In [ ]:
#pass_data_df['Depart_time_epoch'].dt.strftime("%m/%d/%y")
#pass_data_df['Depart_time_epoch']=
(pd.to_datetime(pass_data_df['Depart_time_epoch'],unit='s')) 


In [ ]:
obj1 = obj_1_list_df.merge(from_counts_df, how='outer', on=['Airport_IATA_code', 'Airport_IATA_code'])

In [ ]:
if re.match(r"hello[0-9]+", 'hello1'):
    print('Yes')

In [ ]:
pattern = r"[A-Z][A-Z][A-Z]"

pd.Series(
     ["DEN", "A;S", "AM%", "3b", "03c", "4dx"],
 ).str.contains(pattern)

In [ ]:
#pass_data_df[pass_data_df['From_airport_IATA'].str.match('DEN')== True]
#pass_data_df['From_airport_format']=
pass_data_df['From_airport_IATA'].str.match('[A-Z][A-Z][A-Z]')

In [ ]:
#merging by IATA code
#select remaining from airports
from_values = flight_info_true.From_airport_IATA.unique()
dest_values = flight_info_true.Dest_airport_IATA.unique()
#turn to dataframe
from_values_df = pd.DataFrame(from_values)
dest_values_df = pd.DataFrame(dest_values)
#rename columns to allow merge
from_values_df.columns = ['Airport_IATA_code']
dest_values_df.columns = ['Airport_IATA_code']

In [ ]:
from_merge['From_airport_exists']=from_merge['Airport_name'].str.contains(r"[A-Z]")
dest_merge['Dest_airport_exists']=dest_merge['Airport_name'].str.contains(r"[A-Z]")
#from_merge[from_merge['Airport_name'].str.contains('NaN')==False]
#new_df = df[df["col"].str.contains(word) == False]

In [171]:
import Levenshtein as lev


ModuleNotFoundError: No module named 'Levenshtein'

In [172]:
import pandas as pd
from textdistance import levenshtein

attempts = [['passw0rd', 'pasw0rd'],
            ['passwrd', 'psword'],
            ['psw0rd', 'passwor'],
            ['helloworld', 'heloworl']]

df=pd.DataFrame(attempts, columns=['password', 'attempt'])

df.apply(lambda x: levenshtein.distance(x['password'],  x['attempt']), axis=1)

ModuleNotFoundError: No module named 'textdistance'

In [450]:
#best_match_results_pd[['results','true']] = 
best_match_results_pd['results'].str.split('-',expand=True)
#df[['V','allele']] = df['V'].str.split('-',expand=True)


,0
0,NaN
1,NaN
2,NaN
3,NaN
4,NaN
5,NaN
6,NaN
7,NaN


In [366]:
best_match_results_pd.iloc[1, 0]

['SJD8|75RZ4', 'SJD8775RZ4']

In [176]:
def similar(a, b):
    return SequenceMatcher(None, a, b).ratio()
similar("Apple","Appel")

0.8

In [183]:
round(SequenceMatcher(None, 'WBE6935NU', 'HCA3158QA6').ratio(),3)
round(SequenceMatcher(None, 'UES9151GS5', 'UES915*GS5').ratio(),3)
round(SequenceMatcher(None, 'UES915*GS5', 'UES9151GS5').ratio(),3)


0.9

In [173]:
install python-Levenshtein

SyntaxError: invalid syntax (<ipython-input-173-dfe8d27147ff>, line 1)

In [233]:
f = 'SPRb484HA6'
test_best_true = pd.DataFrame()
for t in true:
    ratio = round(SequenceMatcher(None, t, f).ratio(),3)
    print(t, ratio)
    #test_best_true = test_best_true.append(ratio)


BWI0520BG6 0.1
CDC0302NN5 0.0
CKZ3132BR4 0.2
CXN7304ER2 0.1
CYJ0225CH1 0.1
DAZ3029XA0 0.1
EDV2089LK5 0.1
EZC9678QI6 0.1
HCA3158QA6 0.3
HGO4350KK1 0.1
IEG9308EA5 0.2
JBE2302VO4 0.1
JJM4724RF7 0.2
KKP5277HZ7 0.2
LLZ3798PE3 0.1
MXU9187YC7 0.1
ONL0812DH1 0.2
PAJ3974RK1 0.2
PIT2755XC1 0.1
POP2875LH3 0.3
PUD8209OG3 0.2
SJD8775RZ4 0.3
SPR4484HA6 0.9
UES9151GS5 0.1
VZY2993ME1 0.0
WBE6935NU3 0.1
WTC9125IE5 0.0
WYU2010YH8 0.1
XFG5747ZT9 0.1
YMH6360YP0 0.2


In [466]:
#final_true
#str(final_true)[1:-1] 
#test_best_true
#max_true

"['WYu2010YH8', 'WYU2010YH8']"